# Generate surface slab



In [4]:
from workgraph_collections.ase.common.surface import get_slab_from_miller_indices_ase
from ase.build import bulk
from weas_widget import WeasWidget

pt_bulk = bulk("Pt")
slabs = get_slab_from_miller_indices_ase(pt_bulk, (1, 1, 1), 3,
                                         vacuum=10.0,
                                         center_slab=False)
v = WeasWidget()
v.from_ase(slabs)
v


WeasWidget(children=(BaseWidget(atomScales=[1, 1, 1], atoms={'species': {'Pt': 'Pt'}, 'cell': [2.7718585822512…

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.ase.emt.atomization_energy import atomization_energy

wg = atomization_energy()
wg.to_html()

## Example: Atomization energy of a molecule N2

### Prepare the inputs and submit the workflow


In [3]:
from ase.build import molecule
from ase import Atoms
from workgraph_collections.ase.emt.atomization_energy import atomization_energy
from aiida import load_profile

load_profile()

# create input structure
n_atom = Atoms("N", pbc=True)
n_atom.center(vacuum=5.0)
n2_molecule = molecule("N2", pbc=True)
n2_molecule.center(vacuum=5.0)

metadata = {
    "options": {
        "prepend_text": """
eval "$(conda shell.posix hook)"
conda activate aiida
export OMP_NUM_THREADS=1
        """,
    }
}
# ------------------------- Set the inputs -------------------------
wg = atomization_energy()
wg.tasks["scf_atom"].set(
    {"atoms": n_atom, "computer": "localhost", "metadata": metadata}
)
wg.tasks["scf_mol"].set(
    {"atoms": n2_molecule, "computer": "localhost", "metadata": metadata}
)
wg.tasks["calc_atomization_energy"].set({"molecule": n2_molecule, "computer": "localhost"})
# ------------------------- Submit the calculation -------------------
wg.submit(wait=True, timeout=200)
# ------------------------- Print the output -------------------------
print(
    "Atomization energy:                  {:0.3f} eV".format(
        wg.tasks["calc_atomization_energy"].outputs["result"].value.value
    )
)


ConnectionError: [Errno 111] Connect call failed ('127.0.0.1', 5672)